In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
print(os.listdir("../input/he_challenge_data/data"))

### Need pretrainedmodels to load the pretrained Cedene models into fastai.
### https://github.com/Cadene/pretrained-models.pytorch

In [ ]:
!pip install pretrainedmodels

In [ ]:
from torchvision.models import *
import pretrainedmodels

from fastai import *
from fastai.vision import *
from fastai.vision.models import *
from fastai.vision.learner import model_meta
import fastai

from utils import *
import sys
import torch
fastai.__version__

In [ ]:
lis = os.listdir('../input/he_challenge_data/data/train')

In [ ]:
sub = pd.read_csv('../input/he_challenge_data/data/sample_submission.csv')

In [ ]:
sub.shape

In [ ]:
path = "../input/he_challenge_data/data/train"

In [ ]:
## test filenames to be used to create final submission.
filenames = os.listdir('../input/he_challenge_data/data/test')

In [ ]:
df = pd.read_csv('../input/he_challenge_data/data/train.csv')

In [ ]:
df.head()

In [ ]:
tfms = get_transforms(flip_vert=False,max_zoom=1.0,max_warp=0,do_flip=False,xtra_tfms=[cutout()])
data = (ImageList.from_csv(path, csv_name = '../train.csv', suffix='.jpg')
        .split_by_rand_pct()              
        .label_from_df()            
        .add_test_folder(test_folder = '../test')              
        .transform(tfms, size=400)
        .databunch(num_workers=0,bs=8)) 

tfms1 = get_transforms(flip_vert=False,max_zoom=1.0,max_warp=0,do_flip=False)
data1 = (ImageList.from_csv(path, csv_name = '../train.csv', suffix='.jpg')
        .split_by_rand_pct()              
        .label_from_df()            
        .add_test_folder(test_folder = '../test')              
        .transform(tfms1, size=400)
        .databunch(num_workers=0,bs=8)) 

In [ ]:
## to see the images in train with there labels
data.show_batch(rows=3, figsize=(8,10))

In [ ]:
## print the target classes
print(data.classes)

In [ ]:
learn = cnn_learner(data, models.resnet152, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:
# defaults.device = torch.device('cuda')

## training with one cycle which used cyclic learning rate and learning rate annhelling
learn.fit_one_cycle(2)

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-3))

In [ ]:
preds,_ = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
learn1 = cnn_learner(data, models.densenet161, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:
learn1.fit_one_cycle(3)

In [ ]:
learn1.unfreeze()
learn1.lr_find()
learn1.fit_one_cycle(3, max_lr=slice(1e-6,1e-3))

In [ ]:
preds1,_ = learn1.TTA(ds_type=DatasetType.Test)

In [ ]:
# learn2 = cnn_learner(data, models.resnet101, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:
# learn2.fit_one_cycle(2)

In [ ]:
# learn2.unfreeze()
# learn2.lr_find()
# learn2.fit_one_cycle(2, max_lr=slice(1e-6,1e-3))

In [ ]:
# preds2,_ = learn2.TTA(ds_type=DatasetType.Test)

In [ ]:
learn3 = cnn_learner(data, models.densenet201, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:
learn3.fit_one_cycle(2)

In [ ]:
learn3.unfreeze()
learn3.lr_find()
learn3.fit_one_cycle(2, max_lr=slice(1e-6, 1e-3))

In [ ]:
preds3,_ = learn3.TTA(ds_type=DatasetType.Test)

In [ ]:
learn4 = cnn_learner(data, models.densenet169, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:
learn4.fit_one_cycle(2)

In [ ]:
learn4.unfreeze()
learn4.lr_find()
learn4.fit_one_cycle(2, max_lr=slice(1e-6, 1e-3))

In [ ]:
preds4,_ = learn4.TTA(ds_type=DatasetType.Test)

In [ ]:
# learn5 = cnn_learner(data, models.resnet50, metrics=[error_rate, accuracy], model_dir="/tmp/model/")

In [ ]:
# learn5.fit_one_cycle(2)

In [ ]:
# learn5.unfreeze()
# learn5.lr_find()
# learn5.fit_one_cycle(2, max_lr=slice(1e-6, 1e-3))

In [ ]:
# preds5,_ = learn5.TTA(ds_type=DatasetType.Test)

In [ ]:
## create the submission file 
labelled_preds = []
ensemble_preds = preds + preds1 + preds3 + preds4 # + preds5 + preds2
for pred in ensemble_preds:
    labelled_preds.append(int(np.argmax(pred))+1)

submission = pd.DataFrame(
    {'image_id': filenames,
     'category': labelled_preds,
    })
submission.to_csv('submission.csv',index=False)

In [ ]:
submission.head()

In [ ]:
submission['image_id'] = submission['image_id'].apply(lambda x:x.split('.')[0])

In [ ]:
submission = submission.sort_values(by = ['image_id'], ascending = [True])

In [ ]:
## To download the submission file without Commiting the kernel.

from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "subm_5_learn_2_cycle.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)

In [ ]:
# os.mkdir('saved_models')
# file_path = '../saved_models'
torch.save(learn, 'resnet152.pth')
torch.save(learn1, 'densenet161.pth')
# torch.save(learn2, 'resnet101.pth')
torch.save(learn3, 'densenet201.pth')
torch.save(learn4, 'densenet169.pth')
# torch.save(learn5, 'resnet50.pth')

